In [ ]:
import numpy as np
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
from sklearn.preprocessing import RobustScaler
from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

In [ ]:
import signac
from bo_methods_lib.bo_methods_lib.analyze_data import All_CS_Analysis
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import All_CS_Plotter

#Ignore warnings
import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)

#Set parameters
meth_list = [1, 2, 3, 4, 5, 6, 7]
cs_list = [11,14,2,1,12,13,3,10]
save_csv = False #Set to False if you don't want to save/resave csvs
save_figs = True
bar_modes = ["objs", "time"] #time and/or objs
project = signac.get_project("GPBO_Fix")

analyzer = All_CS_Analysis(cs_list, meth_list, project, "act", save_csv)
plotters = All_CS_Plotter(analyzer, save_figs)

#Get % true found
#Change cs_list here to get averages over select case studies
analyzer.get_percent_true_found(cs_list)

#Make Overall GPBO bar charts
for bmode in bar_modes:
    df_average = plotters.make_bar_charts(bmode)

#Make Derivative Free Bar Charts
df_med_derivfree = plotters.make_derivfree_bar(s_meths = ["NLS", "SHGO-Sob", "NM", "GA"], ver = "med")

In [ ]:
import numpy as np
import pandas as pd
import signac
import os
from itertools import combinations
import signac

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.analyze_data import General_Analysis
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import Plotters

# Ignore warnings
import warnings

warnings.simplefilter("ignore", category=RuntimeWarning)
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)
# from sklearn.exceptions import InconsistentVersionWarning
# warnings.filterwarnings(action='ignore', category=InconsistentVersionWarning)

# Set Stuff
meth_name_val_list = [1, 2, 3, 4, 5, 6, 7]
save_csv = False  # Set to False if you don't want to save/resave csvs
save_figs = True
modes = ["act", "gp", "acq" ] #, "gp", "acq"
project = signac.get_project("GPBO_Fix")

for val in [11,14,2,1,12,13,3,10]:
    criteria_dict = {
        "cs_name_val": val,
        "ep_enum_val": 1,
        "gp_package": "gpflow",
        "meth_name_val": {"$in": meth_name_val_list},
    }
    for mode in modes:
        analyzer = General_Analysis(criteria_dict, project, mode, save_csv)
        plotters = Plotters(analyzer, save_figs)

        ###Get all data from experiments
        df_all_jobs, job_list, theta_true_data = analyzer.get_df_all_jobs(
            criteria_dict, save_csv
        )
        ### Get Best Data from ep experiment
        df_best, job_list_best = analyzer.get_best_data()

        # Loop over z_choices to make comparison line plots
        z_choices = ["sse", "min_sse", "acq"]
        titles = [
            "Min SSE Parameter Values",
            "Min SSE Parameter Values Overall",
            "Optimal Acq Func Parameter Values",
        ]

        # Get best plots for all objectives with all 7 methods on each subplot
        plotters.plot_objs_all_methods(z_choices)

        # Get plot with each method on a different subplot for each obj
        for i in range(len(z_choices)):
            plotters.plot_one_obj_all_methods(z_choices[i])

        # #Can optionally make plots for hyperparameters and theta values
        # for i in range(len(job_list_best)):
        #     #Plot hyperparameters
            # plotters.plot_hypers(job_list_best[i])

        #     #Plot param values at min_sse, the best theta_values of min_sse overall, and param values at max ei
        #     for j in range(len(z_choices)):
        #         plotters.plot_thetas(job_list_best[i], z_choices[j], title = titles[j])

In [ ]:
import numpy as np
import signac
from itertools import combinations
from bo_methods_lib.bo_methods_lib.analyze_data import General_Analysis
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import Plotters

# Ignore warnings
import warnings

warnings.simplefilter("ignore", category=RuntimeWarning)
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)

meth_name_val_list = [1, 2, 3]
save_csv = False  # Set to False if you don't want to save/resave csvs
save_figs = False
modes = ["act"]
project = signac.get_project("GPBO_Fix")

for val in [1]:
    criteria_dict = {
        "cs_name_val": val,
        "ep_enum_val": 1,
        "gp_package": "gpflow",
        "meth_name_val": {"$in": meth_name_val_list},
    }

    for mode in modes:
        analyzer = General_Analysis(criteria_dict, project, mode, save_csv)
        plotters = Plotters(analyzer, save_figs)

        # Get Best Data from ep experiment
        ### Get Best Data from ep experiment
        df_best, job_list_best = analyzer.get_best_data()

        # Set z_choices and levels
        z_choices = ["sse_sim", "sse_mean", "sse_var", "acq"]
        levels = [100, 100, 100, 100]

        # Loop over best jobs
        for i in range(len(job_list_best)):
            # Get jobs, runs, and iters to examine
            job = job_list_best[i]
            run_num = int(df_best["Run Number"].iloc[i])
            bo_iter = int(df_best["BO Iter"].iloc[i])

            # Back out number of parameters
            string_val = df_best["Theta Min Obj"].iloc[0]
            try:
                numbers = [
                    float(num)
                    for num in string_val.replace("[", "").replace("]", "").split()
                ]
            except:
                numbers = [float(num) for num in string_val]

            # Create list of parameter pair combinations
            dim_theta = len(np.array(numbers).reshape(-1, 1))
            dim_list = np.linspace(0, dim_theta - 1, dim_theta)
            pairs = len((list(combinations(dim_list, 2))))

            # Loop over parameter pairs
            if pairs < 3:
                for pair in range(pairs):
                    plotters.plot_hms_gp_compare(
                        job, run_num, bo_iter, pair, z_choices, levels
                    )


In [ ]:
# Define Simulator Class (Export your Simulator Object Here)
simulator = simulator_helper_test_fxns(1,0,0, 1)

# Generate Exp Data (OR Add your own experimental data as a Data class object)
set_seed = 1  # Set set_seed to 1 for data generation
gen_meth_x = Gen_meth_enum(2)
num_pts = 100
exp_data = simulator.gen_exp_data(num_pts, gen_meth_x, 1)

# Set simulator noise_std artifically as 5% of y_exp mean (So that noise will be set rather than trained)
simulator.noise_std = np.abs(np.mean(exp_data.y_vals)) * 0.05

In [ ]:
print(exp_data.y_vals)

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and axis
fig, ax = plt.subplots()

# Plot the data
ax.plot(exp_data.x_vals, exp_data.y_vals, color='blue', linestyle='-', linewidth=2)

# Add labels and title
ax.set_xlabel('X-Value')
ax.set_ylabel('Y-Value')
ax.set_title('BOD Curve Example')

# Add grid
ax.grid(True)

# Add legend
# ax.legend()

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Generate sample 2D data
X, Y = exp_data.x_vals.reshape(num_pts, num_pts,2).T
Z = exp_data.y_vals.reshape(X.shape)

# Create the plot
fig, ax = plt.subplots()

# Plot filled contours with contourf
contour_filled = ax.contourf(X, Y, Z, levels=20, cmap='jet')

# Add contour lines
contour_lines = ax.contour(X, Y, Z, levels=20, colors='black', linestyle = '--', linewidths=0.5)

# Add a colorbar for the filled contours
cbar = fig.colorbar(contour_filled, ax=ax)
cbar.set_label('Magnitude')

# Add labels to the contour lines
ax.clabel(contour_lines, inline=True, fontsize=8)

# Set axis labels
ax.set_xlabel('X1')
ax.set_ylabel('X2')

# Title
ax.set_title('Müller ' + r'$x_0$' ' Data')

# Show the plot
plt.show()
